In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from IPython.display import Audio, display
import torch
import torchaudio

from api import StableTTSAPI
from pathlib import Path

device = 'cuda' if torch.cuda.is_available() else 'cpu'


tts_model_path = '/data/lxn/StableTTS_24k_bk/checkpoints_6seg_sbd/checkpoint.pt'
vocoder_model_path = '/data/lxn/StableTTS_24k/BigVGAN/bigvgan_generator.pt' # path to vocoder checkpoint
vocoder_config_path = '/data/lxn/StableTTS_24k/BigVGAN/configs/bigvgan_v2_24khz_100band_256x.json'

model = StableTTSAPI(tts_model_path, vocoder_model_path, vocoder_config_path)
model.to(device)

tts_param, vocoder_param = model.get_params()
print(f'tts_param: {tts_param}, vocoder_param: {vocoder_param}')


Removing weight norm...
tts_param: 31.533357, vocoder_param: 112.414512


In [ ]:
text = 'According to a spokesperson for Springfield City Schools, Fulton Elementary School was evacuated and taken to Springfield High School.'
ref_audio = 'PATH_TO_REF_AUDIO'
language = 'english' # support chinese, japanese and english
solver = 'euler' # recommend using euler, midpoint or dopri5
steps = 15
cfg = 2 # recommend 1-4

audio_output, mel_output = model.inference(text, ref_audio, language, steps, 1, 1, solver, cfg)

y, sr = torchaudio.load(ref_audio)
if sr != 24000:
    y = torchaudio.functional.resample(y, sr, 24000)

display(Audio(y, rate=model.mel_config.sampling_rate))
display(Audio(audio_output, rate=model.mel_config.sampling_rate))